# カーネルトリック

このノートブックでは、SVMのカーネルトリックについて学習します。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles, make_moons
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)

# 非線形分離可能なデータの生成
X_circles, y_circles = make_circles(n_samples=100, noise=0.1, factor=0.3, random_state=42)
X_moons, y_moons = make_moons(n_samples=100, noise=0.1, random_state=42)

# データの標準化
X_circles_scaled = StandardScaler().fit_transform(X_circles)
X_moons_scaled = StandardScaler().fit_transform(X_moons)

# 異なるカーネルでのSVM
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
datasets = [('Circles', X_circles_scaled, y_circles), ('Moons', X_moons_scaled, y_moons)]

fig, axes = plt.subplots(2, 4, figsize=(20, 10))

for i, (dataset_name, X, y) in enumerate(datasets):
    for j, kernel in enumerate(kernels):
        # SVMの訓練
        svm = SVC(kernel=kernel, random_state=42)
        svm.fit(X, y)
        
        # 予測
        y_pred = svm.predict(X)
        accuracy = accuracy_score(y, y_pred)
        
        # 決定境界の可視化
        h = 0.02
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
        
        Z = svm.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        
        axes[i, j].contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.RdYlBu)
        axes[i, j].scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolors='black')
        axes[i, j].set_title(f'{dataset_name} - {kernel.capitalize()} (Acc: {accuracy:.3f})')
        axes[i, j].set_xlabel('Feature 1')
        axes[i, j].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

# カーネル関数の可視化
def rbf_kernel(x1, x2, gamma=1.0):
    """RBFカーネル関数"""
    return np.exp(-gamma * np.linalg.norm(x1 - x2)**2)

def poly_kernel(x1, x2, degree=3, gamma=1.0, coef0=0):
    """多項式カーネル関数"""
    return (gamma * np.dot(x1, x2) + coef0) ** degree

def sigmoid_kernel(x1, x2, gamma=1.0, coef0=0):
    """シグモイドカーネル関数"""
    return np.tanh(gamma * np.dot(x1, x2) + coef0)

# カーネル関数の比較
x1 = np.array([1, 2])
x2 = np.array([3, 4])

print("=== カーネル関数の比較 ===")
print(f"RBFカーネル: {rbf_kernel(x1, x2):.4f}")
print(f"多項式カーネル: {poly_kernel(x1, x2):.4f}")
print(f"シグモイドカーネル: {sigmoid_kernel(x1, x2):.4f}")

# 異なるパラメータでの性能
gamma_values = [0.1, 1, 10, 100]
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, gamma in enumerate(gamma_values):
    svm_gamma = SVC(kernel='rbf', gamma=gamma, random_state=42)
    svm_gamma.fit(X_circles_scaled, y_circles)
    y_pred = svm_gamma.predict(X_circles_scaled)
    accuracy = accuracy_score(y_circles, y_pred)
    
    # 決定境界の可視化
    h = 0.02
    x_min, x_max = X_circles_scaled[:, 0].min() - 1, X_circles_scaled[:, 0].max() + 1
    y_min, y_max = X_circles_scaled[:, 1].min() - 1, X_circles_scaled[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    Z = svm_gamma.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    axes[i].contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.RdYlBu)
    axes[i].scatter(X_circles_scaled[:, 0], X_circles_scaled[:, 1], c=y_circles, cmap=plt.cm.RdYlBu, edgecolors='black')
    axes[i].set_title(f'RBF Kernel (γ={gamma}) - Accuracy: {accuracy:.3f}')
    axes[i].set_xlabel('Feature 1')
    axes[i].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

print(f"\n=== 異なるγ値での性能 ===")
for gamma in gamma_values:
    svm_gamma = SVC(kernel='rbf', gamma=gamma, random_state=42)
    svm_gamma.fit(X_circles_scaled, y_circles)
    y_pred = svm_gamma.predict(X_circles_scaled)
    accuracy = accuracy_score(y_circles, y_pred)
    print(f"γ={gamma}: 精度={accuracy:.4f}, サポートベクトル数={len(svm_gamma.support_vectors_)}")
